### Training data preparation for Yolov5 bounding box detection

Pre-requisites:

- ensure you have download the DICOM data (download_data_lidc.ipynb)

In [ ]:
!pip install -r requirements.txt

In [1]:
import pylidc as pl
from pylidc.utils import consensus
import pydicom as dicom
from skimage.measure import find_contours
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from PIL import Image
import contextlib
import matplotlib.patches as patches
import os
import imageio
import sys
from glob import glob
import os
import cv2
import numpy as np
from skimage import morphology
from skimage import measure
from sklearn.cluster import KMeans
from skimage.transform import resize
import os
from numpy import random
import time
import pandas as pd
from PIL import Image
import contextlib
import os
import contextlib
import imageio
from __future__ import print_function
import sys
from contextlib import redirect_stdout, redirect_stderr

#### Routines for bounding box extraction

Define some routines to get the bounding around for the masks for the image

In [2]:
import matplotlib.patches as patches
def get_bounding_box(img):
    minx = []
    miny = []
    maxx = []
    maxy = []
    contours = find_contours(img.astype(float), 0.5)
    for contour in contours:
        minx.append(np.min(contour[:,1]))
        maxx.append(np.max(contour[:,1]))
        miny.append(np.min(contour[:,0]))
        maxy.append(np.max(contour[:,0]))
        
    if len(minx) > 0:
        return [(a, b, w-a, h-b) for a, b, w, h in zip(minx, miny, maxx, maxy)]
    else:
        return None
    
def plot_bounding_box(img):
    fig, ax = plt.subplots()
    ax.imshow(img, cmap=plt.cm.gray)
    
    boxes = get_bounding_box(img)
    if boxes:
        for box in boxes:
            x,y,w,h = box
            print("x = {}, y = {}, w = {}, h= {}".format(x,y,w,h))
            rect = patches.Rectangle((x, y), w, h, linewidth=1, edgecolor='r', facecolor='none')
            ax.add_patch(rect)

Routine to find all bounding boxes for the mask and plotting

In [3]:
def find_all_bboxes(nodule_annotation, nodule_id, plot=False):
    bboxes = []
    #cmask, cbbox, masks = consensus(nodule_annotation)
    
    cmask_CT = []
    cbbox_CT = []
    masks_CT = []
    for nodule_idx, nodule in enumerate(nodules_annotation):
        cmask, cbbox, masks = consensus(nodule)
        cmask_CT.append(cmask)
        cbbox_CT.append(cbbox)
        masks_CT.append(masks)
    CT_mask = np.zeros_like(vol)    
    nodule_num = len(cmask_CT)
    print("Number of nodules for patient {} is {}".format(patient_id, nodule_num))
    
    for i in range(nodule_num):
        cmask = cmask_CT[i]
        cbbox = cbbox_CT[i]
        masks = masks_CT[i]
        CT_mask[cbbox] += cmask
        

    print("There are {} slices in the image".format(cbbox_CT[nodule_id][2].stop - cbbox_CT[nodule_id][2].start))
    for idx in np.arange(cbbox_CT[nodule_id][2].start, cbbox_CT[nodule_id][2].stop):
        boxes = get_bounding_box(CT_mask[:,:,idx])
        if plot:
            plot_bounding_box(CT_mask[:,:,idx])
        bboxes.append(boxes)
    return bboxes

Routine to plot the bounding box around the mask in the full size mask

In [4]:
def plot_bbox_on_full_size_mask(vol, nodules_annotation, nodule_id):
    cmask_CT = []
    cbbox_CT = []
    masks_CT = []
    for nodule_idx, nodule in enumerate(nodules_annotation):
        cmask, cbbox, masks = consensus(nodule)
        cmask_CT.append(cmask)
        cbbox_CT.append(cbbox)
        masks_CT.append(masks)
    CT_mask = np.zeros_like(vol)    
    nodule_num = len(cmask_CT)
    print("Number of nodules for patient is {}".format(nodule_num))
    
    for i in range(nodule_num):
        cmask = cmask_CT[i]
        cbbox = cbbox_CT[i]
        masks = masks_CT[i]
        CT_mask[cbbox] += cmask
        
    for slice_idx in np.arange(cbbox_CT[nodule_id][2].start, cbbox_CT[nodule_id][2].stop):
        plot_bounding_box(CT_mask[:,:,slice_idx])

Routine to plot the bounding box around the nodule using the mask bounding box coordinates

In [5]:
def plot_bbox_on_image(vol, nodules_annotation, nodule_id):
    cmask_CT = []
    cbbox_CT = []
    masks_CT = []
    for nodule_idx, nodule in enumerate(nodules_annotation):
        cmask, cbbox, masks = consensus(nodule)
        cmask_CT.append(cmask)
        cbbox_CT.append(cbbox)
        masks_CT.append(masks)
    CT_mask = np.zeros_like(vol)    
    nodule_num = len(cmask_CT)
    print("Number of nodules for patient is {}".format(nodule_num))
    
    for i in range(nodule_num):
        cmask = cmask_CT[i]
        cbbox = cbbox_CT[i]
        masks = masks_CT[i]
        CT_mask[cbbox] += cmask
        
    for slice_idx in np.arange(cbbox_CT[nodule_id][2].start, cbbox_CT[nodule_id][2].stop):
        boxes = get_bounding_box(CT_mask[:,:,slice_idx])
        img = vol[:,:,slice_idx]
        fig, ax = plt.subplots()
        ax.imshow(img, cmap=plt.cm.gray)
        if boxes:
            for box in boxes:
                x, y, w, h = box
                print("x = {}, y = {}, w = {}, h= {}".format(x,y,w,h))
                rect = patches.Rectangle((x, y), w, h, linewidth=1, edgecolor='r', facecolor='none')
                ax.add_patch(rect)

### Prepare the dataset for YoloV5 training

Routine to create the image and the labels (based on bounding boxes) for Yolov5 training

#### Clear the contents of the generated images and labels for Yolov5

In [6]:
import shutil
def clear_dirs(paths):
    for path in paths:
        if os.path.isdir(path):
            shutil.rmtree(path)

##### For all the patients we downloaded, extract the bounding box and create the image and labels for YoloV5 training

### Save image label and mask

- For each CT image, save the image, the mask and the bounding box label
- For bounding box dataset, save empty files for images with no mask.

Routine to suppress stdout and stderr messages

In [7]:
class RedirectStdStreams(object):
    def __init__(self, stdout=None, stderr=None):
        self._stdout = stdout or sys.stdout
        self._stderr = stderr or sys.stderr

    def __enter__(self):
        self.old_stdout, self.old_stderr = sys.stdout, sys.stderr
        self.old_stdout.flush(); self.old_stderr.flush()
        sys.stdout, sys.stderr = self._stdout, self._stderr

    def __exit__(self, exc_type, exc_value, traceback):
        self._stdout.flush(); self._stderr.flush()
        sys.stdout = self.old_stdout
        sys.stderr = self.old_stderr

In [8]:
def un(a, b):
    return np.union1d(a, b)

def unr(ll, acc, i):
    if i < len(ll):
        j = i + 1
        acc = un(ll[i], acc)
        return unr(ll, acc, j)
    else:
        return acc

def get_slices_with_masks(cbbox_CT):
    ll = []
    for cb in cbbox_CT:
        ll.append(np.arange(cb[2].start, cb[2].stop+1))
    return unr(ll, [], 0)

#### Preprocessing routine for images

In [9]:
#code ref: https://github.com/JenifferWuUCLA/pulmonary-nodules-segmentation/blob/master/tianchi_segment_lung_ROI.ipynb
def preprocess_image(img):
    mean = np.mean(img)
    std = np.std(img)
    img = img-mean
    img = img/std
    # Find the average pixel value near the lungs
    #　to renormalize washed out images
    #middle = img[100:400, 100:400]
    middle = img
    mean = np.mean(middle)
    max = np.max(img)
    min = np.min(img)
    # To improve threshold finding, I'm moving the
    #　underflow and overflow on the pixel spectrum
    img[img==max]=mean
    img[img==min]=mean
    # Using Kmeans to separate foreground (radio-opaque tissue)
    #　and background (radio transparent tissue ie lungs)
    # Doing this only on the center of the image to avoid
    #　the non-tissue parts of the image as much as possible
    kmeans = KMeans(n_clusters=2).fit(np.reshape(middle, [np.prod(middle.shape), 1]))
    centers = sorted(kmeans.cluster_centers_.flatten())
    threshold = np.mean(centers)
    thresh_img = np.where(img<threshold, 1.0, 0.0)  # threshold the image
    eroded = morphology.erosion(thresh_img, np.ones([4, 4]))
    dilation = morphology.dilation(eroded, np.ones([10, 10]))
    labels = measure.label(dilation)
    regions = measure.regionprops(labels)
    good_labels = []
    for prop in regions:
        B = prop.bbox
        if B[2]-B[0] < 475 and B[3]-B[1] < 475 and B[0] > 40 and B[2] < 472:
            good_labels.append(prop.label)

    mask = np.zeros_like(labels)
    for N in good_labels:
        mask = mask + np.where(labels == N, 1, 0)

    mask = morphology.dilation(mask, np.ones([10, 10]))  # one last dilation
    return img * mask

##### Preprocessing steps:

We will save  512x512 images and the corresponding labels in Yolov5 pytorch format 

- Mark the mask for nodules which were annotated by all 4 radiologists
- Select all the slices which have annotated masks
- Select an equal number of slices which have no annotated masks (for negative images)
- Create the bounding box label for each slice with a mask using find coutours method
- Process the image to highlight the region of interest
- Save the processed image and the labels

In [10]:
"""
Save images and labels for nodules that have annotations from all 4 radiologists
"""
def save_image_mask_label(patient_id, imagedirname, labeldirname, maskdirname, normalize=False):
    
    # For supressing stdout and stderr messages
    devnull = open(os.devnull, 'w')
    
    # Read the volume (3D-dicom image) for the patient
    scan = pl.query(pl.Scan).filter(pl.Scan.patient_id.in_([patient_id]))[0]
    nodules_annotation = scan.cluster_annotations()
    try:
        with contextlib.redirect_stdout(None):
            vol = scan.to_volume()
    except:
        print("Error loading DICOM for patient {}".format(patient_id))
        return
    
    # Process the mask for the image
    cmask_CT = []
    cbbox_CT = []
    masks_CT = []
    for nodule_idx, nodule in enumerate(nodules_annotation):
        if len(nodule) >=4:
            cmask, cbbox, masks = consensus(nodule)
            cmask_CT.append(cmask)
            cbbox_CT.append(cbbox)
            masks_CT.append(masks)
        
    if len(cbbox_CT) <=0:
        print("No relevant nodules for patient {}".format(patient_id))
        return
    
    CT_mask = np.zeros_like(vol)    
    nodule_num = len(cmask_CT)
      
    for i in range(nodule_num):
        cmask = cmask_CT[i]
        cbbox = cbbox_CT[i]
        masks = masks_CT[i]
        CT_mask[cbbox] += cmask
    
    # Get the slices that have annotations
    slices_with_mask = get_slices_with_masks(cbbox_CT)

    idxs = np.arange(CT_mask.shape[2])
    
    # Get all slices that have no annotations
    compl_idxs = [idx for idx in idxs if idx not in slices_with_mask]
    SEED = 2021
    np.random.seed(SEED)
    # Random sample the slices with no annotations
    np.random.shuffle(compl_idxs)
    # Select equal number of non annotated slices as the number of annotated slices
    compl_idxs = compl_idxs[:len(slices_with_mask)]
    all_slices = np.union1d(slices_with_mask, compl_idxs)
    # Save the images and masks and labels for only slices_with_mask and compl_idxs
    all_slices = all_slices.astype(int)

    # Iterate through all slices and save the images
    z_slice_cnt = 0
    for z_slice in all_slices:
        im = vol[:,:,z_slice]
        # mean normalize the image to prevent unit8 conversion errors when saving with imageio
        if normalize:
            im = im - np.mean(im)
            im = im / np.std(im)
        try:
            # in order to preserve the richness of the image, imageio is used to save and then read the image back from the disk
            # TODO: could potentially do it in memory as well
            # TODO: create the temp folder
            imageio.imwrite('/data/unet/temp/temp_img.jpg', im)
            disk_img = cv2.imread('/data/unet/temp/temp_img.jpg')
        except:
            continue
        
        im = cv2.cvtColor(disk_img, cv2.COLOR_BGR2GRAY)
        img_t = preprocess_image(im)
        
        image_filename = "{}_{}.jpg".format(patient_id, z_slice)
        with RedirectStdStreams(stdout=devnull, stderr=devnull):
            imageio.imwrite(imagedirname + "/" + image_filename, img_t)
        z_slice_cnt += 1
    
    # Save bounding box labels
    for k_slice in all_slices:
        msk = CT_mask[:,:,k_slice]
        boxes = get_bounding_box(msk)
        if boxes:
            for box in boxes:
                x, y, w, h = box
                # Drop small objects
                if (w <= 3) or (h <= 3):
                    continue
                center_x = (x + ((w)/2))/vol[:,:,k_slice].shape[0]
                center_y = (y + ((h)/2))/vol[:,:,k_slice].shape[1]
                width = w/vol[:,:,k_slice].shape[0]
                height = h/vol[:,:,k_slice].shape[1]

                c = 0 # there is only one class, the 'nodule'
                # Save this as the label for the image for the YoloV5 object detection training
                label = "{} {} {} {} {}".format(c, center_x, center_y, width, height)
                label_filename = "{}_{}.txt".format(patient_id, k_slice)

                f = open(labeldirname + "/" + label_filename, "a")
                f.write(label + "\n")
                f.close()
        else:
            # Save this as the label for the image for the YoloV5 object detection training
            label_filename = "{}_{}.txt".format(patient_id, k_slice)
            f = open(labeldirname + "/" + label_filename, "a")
            f.write("")
            f.close()

In [11]:
def create_dataset(patient_list, dicom_dir, root_data_dir, dtype='train'):
    # Set the pylidc data config
    print("Setting DICOM dir = {}".format(dicom_dir))
    f = open ('/root/.pylidcrc','w')    #For Colab
    f.write('[dicom]'+'\n')
    f.write('path =' + dicom_dir +'\n')
    f.write('warn = True')
    f.close()
    
    image_path = root_data_dir + '/' + dtype + '/images'
    mask_path = root_data_dir + '/' + dtype + '/masks'
    label_path = root_data_dir + '/' + dtype + '/labels'
    
    for idx, patient in tqdm(enumerate(patient_list)):
        if not os.path.exists(image_path):
            print("Removing and then creating directory {}".format(image_path))
            os.makedirs(image_path)
        if not os.path.exists(label_path):
            print("Removing and then creating directory {}".format(label_path))
            os.makedirs(label_path)
        if not os.path.exists(mask_path):
            print("Removing and then creating directory {}".format(mask_path))
            os.makedirs(mask_path)
        save_image_mask_label(patient, image_path, label_path, mask_path, normalize=True)

In [ ]:
patient_list_train = os.listdir('/data/lidc/dicom/train/sorted_gcsfiles/')
print("Train patients = {}".format(len(patient_list_train)))
patient_list_test = os.listdir('/data/lidc/dicom/test/sorted_gcsfiles/')
print("Test patients = {}".format(len(patient_list_test)))
patient_list_val = os.listdir('/data/lidc/dicom/val/sorted_gcsfiles/')
print("Val patients = {}".format(len(patient_list_val)))

In [48]:
%%time
create_dataset(patient_list_train, '/data/lidc/dicom/train/sorted_gcsfiles/', '/data/yolov5/processed/', 'train')

Setting DICOM dir = /rawdata/train/sorted_gcsfiles


1it [00:01,  1.72s/it]

No relevant nodules for patient LIDC-IDRI-0971


2it [00:05,  2.95s/it]

No relevant nodules for patient LIDC-IDRI-0856


...

No relevant nodules for patient LIDC-IDRI-0224


Lossy conversion from float64 to uint8. Range [-1.5570604896196674, 3.681015976447453]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [-1.5526020781433791, 3.368374540103394]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [-1.5527212027368027, 3.738373610278849]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [-1.5548307855075543, 3.4325168410474785]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [-1.5581292619202654, 3.416922669705949]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [-1.5538057923818374, 5.201858154304604]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [-1.5655979817778662, 4.18549200670

No relevant nodules for patient LIDC-IDRI-0830


696it [1:39:27,  8.57s/it]

No relevant nodules for patient LIDC-IDRI-0374
CPU times: user 3h 17min 37s, sys: 9min 11s, total: 3h 26min 48s
Wall time: 1h 39min 27s


In [49]:
%%time
create_dataset(patient_list_val, '/data/lidc/dicom/val/sorted_gcsfiles/', '/data/yolov5/processed', 'val')

Setting DICOM dir = /rawdata/yolov5/val/sorted_gcsfiles/


1it [00:02,  2.04s/it]

No relevant nodules for patient LIDC-IDRI-0026


2it [00:02,  1.37s/it]

No relevant nodules for patient LIDC-IDRI-0471


...

No relevant nodules for patient LIDC-IDRI-0933


Lossy conversion from float64 to uint8. Range [-1.6649053128725158, 2.4561546823450966]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [-1.574942818757396, 3.058865988227176]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [-1.579894686336951, 3.1365756377418976]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [-1.5800703563006537, 3.3486232557129068]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [-1.5455274194482853, 4.7169276188531]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [-1.5372879437530933, 4.778069823693188]. Convert image to uint8 prior to saving to suppress this warning.
152it [16:00,  3.38s/it]

No relevant nodules for patient LIDC-IDRI-0542


Lossy conversion from float64 to uint8. Range [-1.777103575545145, 1.993254376849707]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [-1.7775073454482377, 1.956287664015495]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [-1.776254431830915, 1.9833764449514064]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [-1.7761867175482011, 1.9448891819650427]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [-1.7776868693111223, 2.2237195610614364]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [-1.7780048901580985, 2.002361413700077]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [-1.77828402961677, 2.0604993138182

No relevant nodules for patient LIDC-IDRI-0609


In [50]:
%%time
create_dataset(patient_list_test, '/data/lidc/dicom/test/sorted_gcsfiles/', '/data/yolov5/processed', 'test')

Setting DICOM dir = /data/yolov5/test/sorted_gcsfiles/


0it [00:00, ?it/s]

Removing and then creating directory /data/yolov5/processed/test/images
Removing and then creating directory /data/yolov5/processed/test/labels
Removing and then creating directory /data/yolov5/processed/test/masks


Lossy conversion from float64 to uint8. Range [-1.824123437780138, 1.6709841419516223]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [-1.8167267574718753, 1.7633115301301996]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [-1.816170989263953, 1.798384558270679]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [-1.790191622847736, 2.037693850057179]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [-1.7892944349762916, 1.9777792193095314]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [-1.7872577273167005, 1.9310285022747031]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [-1.7826421430719488, 1.96915267101

No relevant nodules for patient LIDC-IDRI-0395


...

No relevant nodules for patient LIDC-IDRI-0028


Lossy conversion from float64 to uint8. Range [-1.2759251474821955, 4.117580914485856]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [-1.2800200170768408, 4.143794228062117]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [-1.2794686008464071, 4.339217218910107]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [-0.8553912854179869, 5.001421470803144]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [-0.8573803084921696, 4.855166170881438]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [-0.8615006832691169, 4.859915416183533]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [-0.8628151992924129, 4.971027449466

No relevant nodules for patient LIDC-IDRI-0564


48it [06:29,  8.11s/it]

No relevant nodules for patient LIDC-IDRI-0125
